In [ ]:
# تثبيت المكتبات
!pip install pose-format opencv-python Pillow vidgear

In [1]:
from pose_format import Pose
from pose_format.pose_visualizer import PoseVisualizer, FastAndUglyPoseVisualizer
import numpy as np

## 📥 تحميل ملف Pose للعرض

In [2]:
def load_pose(pose_path: str) -> Pose:
    """تحميل ملف pose"""
    with open(pose_path, 'rb') as f:
        pose = Pose.read(f.read())
    print(f"✅ تم تحميل: {pose_path}")
    print(f"   الإطارات: {pose.body.data.shape[0]}")
    print(f"   FPS: {pose.body.fps}")
    return pose



In [3]:
FILE_PATH = "./test/pose_files/ESL_161_ESL_كتب_كتاب.pose"
pose = load_pose(FILE_PATH)

✅ تم تحميل: ./test/pose_files/ESL_161_ESL_كتب_كتاب.pose
   الإطارات: 52
   FPS: 30.0


## 1️⃣ استخدام CLI للعرض السريع

In [4]:
# عرض pose كفيديو
!visualize_pose -i "path/to/your/file.pose" -o "output_video.mp4"

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\micrk\miniconda3\envs\ML_sign_language\Scripts\visualize_pose.exe\__main__.py", line 6, in <module>
  File "C:\Users\micrk\miniconda3\envs\ML_sign_language\Lib\site-packages\pose_format\bin\pose_visualizer.py", line 35, in main
    raise FileNotFoundError(f"Pose file {args.i} not found")
FileNotFoundError: Pose file path/to/your/file.pose not found


In [ ]:
# مع التطبيع (ليظهر بشكل أفضل)
!visualize_pose -i "path/to/your/file.pose" -o "output_normalized.mp4" --normalize

## 2️⃣ إنشاء فيديو من Pose

In [5]:
def create_video_from_pose(pose: Pose, output_path: str, 
                            background_color: tuple = (255, 255, 255),
                            normalize: bool = True,
                            max_frames: int = None):
    """
    إنشاء فيديو من ملف pose
    
    Parameters:
    -----------
    pose : Pose
        كائن الـ Pose
    output_path : str
        مسار الفيديو الناتج
    background_color : tuple
        لون الخلفية (RGB)
    normalize : bool
        تطبيع الـ pose للعرض الأفضل
    max_frames : int, optional
        الحد الأقصى للإطارات
    """
    # نسخة للتعديل
    pose_copy = pose.copy()
    
    if normalize:
        # التأكد من أن الـ pose يبدأ من (0,0) ويملأ المساحة
        pose_copy.focus()
    
    # إنشاء visualizer
    visualizer = PoseVisualizer(pose_copy)
    
    # إنشاء الإطارات
    frames = visualizer.draw(
        background_color=background_color,
        max_frames=max_frames
    )
    
    # حفظ كفيديو
    visualizer.save_video(output_path, frames)
    
    print(f"✅ تم حفظ الفيديو: {output_path}")



In [ ]:
SAVE_DIR  = "./test/"
create_video_from_pose(pose, "output.mp4", normalize=False)

0it [00:00, ?it/s]WARNING:root:Video shape is not divisible by 2. Can not use H.264. Consider resizing to a divisible shape.
52it [00:55,  1.07s/it]


✅ تم حفظ الفيديو: output.mp4


In [ ]:
def create_video_with_colors(pose: Pose, output_path: str):
    """
    إنشاء فيديو بألوان مختلفة للخلفية
    """
    colors = {
        'white': (255, 255, 255),
        'black': (0, 0, 0),
        'green': (0, 255, 0),
        'blue': (0, 0, 255),
        'gray': (128, 128, 128)
    }
    
    for name, color in colors.items():
        output = output_path.replace('.mp4', f'_{name}.mp4')
        create_video_from_pose(pose, output, background_color=color)

# create_video_with_colors(pose, "pose_video.mp4")

## 3️⃣ إنشاء GIF متحرك

In [ ]:
def create_gif_from_pose(pose: Pose, output_path: str,
                         background_color: tuple = (255, 255, 255),
                         max_frames: int = None,
                         normalize: bool = True):
    """
    إنشاء GIF من pose
    
    ملاحظة: GIF أكبر حجماً من الفيديو، استخدمه للعرض السريع فقط
    """
    pose_copy = pose.copy()
    
    if normalize:
        pose_copy.focus()
    
    visualizer = PoseVisualizer(pose_copy)
    
    frames = list(visualizer.draw(
        background_color=background_color,
        max_frames=max_frames
    ))
    
    visualizer.save_gif(output_path, frames)
    
    print(f"✅ تم حفظ GIF: {output_path}")

# create_gif_from_pose(pose, "output.gif", max_frames=100)

In [ ]:
def create_gif_in_memory(pose: Pose, max_frames: int = 50) -> bytes:
    """
    إنشاء GIF في الذاكرة (بدون حفظ كملف)
    مفيد للويب وواجهات المستخدم
    """
    pose_copy = pose.copy()
    pose_copy.focus()
    
    visualizer = PoseVisualizer(pose_copy)
    frames = list(visualizer.draw(max_frames=max_frames))
    
    # None تعني حفظ في الذاكرة وإرجاع bytes
    gif_bytes = visualizer.save_gif(None, frames)
    
    print(f"✅ تم إنشاء GIF في الذاكرة: {len(gif_bytes)} bytes")
    return gif_bytes

# gif_data = create_gif_in_memory(pose)

## 4️⃣ إنشاء PNG متحرك (شفاف)

In [ ]:
def create_transparent_png(pose: Pose, output_path: str, max_frames: int = None):
    """
    إنشاء PNG متحرك بخلفية شفافة
    """
    pose_copy = pose.copy()
    pose_copy.focus()
    
    visualizer = PoseVisualizer(pose_copy)
    
    # transparency=True للشفافية
    frames = list(visualizer.draw(
        background_color=(255, 255, 255),
        max_frames=max_frames,
        transparency=True
    ))
    
    visualizer.save_png(output_path, frames, transparency=True)
    
    print(f"✅ تم حفظ PNG شفاف: {output_path}")

# create_transparent_png(pose, "transparent_pose.png")

## 5️⃣ رسم Pose على فيديو موجود (Overlay)

In [ ]:
def overlay_pose_on_video(pose: Pose, video_path: str, output_path: str,
                          blur_background: bool = False):
    """
    رسم الـ pose فوق فيديو موجود
    
    Parameters:
    -----------
    pose : Pose
        كائن الـ Pose
    video_path : str
        مسار الفيديو الأصلي
    output_path : str
        مسار الفيديو الناتج
    blur_background : bool
        تطبيق blur على الخلفية
    """
    visualizer = PoseVisualizer(pose)
    
    # رسم على الفيديو
    frames = visualizer.draw_on_video(
        background_video=video_path,
        blur=blur_background
    )
    
    visualizer.save_video(output_path, frames)
    
    print(f"✅ تم حفظ الفيديو مع overlay: {output_path}")

# ملاحظة: FPS الـ pose يجب أن يطابق FPS الفيديو
# overlay_pose_on_video(pose, "original_video.mp4", "with_skeleton.mp4")

## 6️⃣ حفظ إطار واحد كصورة

In [ ]:
def save_single_frame(pose: Pose, frame_index: int, output_path: str,
                      background_color: tuple = (255, 255, 255)):
    """
    حفظ إطار واحد كصورة
    """
    pose_copy = pose.copy()
    pose_copy.focus()
    
    visualizer = PoseVisualizer(pose_copy)
    
    # الحصول على إطار محدد
    frames = list(visualizer.draw(
        background_color=background_color,
        max_frames=frame_index + 1
    ))
    
    if frame_index < len(frames):
        visualizer.save_frame(output_path, frames[frame_index])
        print(f"✅ تم حفظ الإطار {frame_index}: {output_path}")
    else:
        print(f"❌ الإطار {frame_index} غير موجود (الحد الأقصى: {len(frames)-1})")

# save_single_frame(pose, 0, "first_frame.png")   # أول إطار
# save_single_frame(pose, 50, "frame_50.png")    # الإطار 50

In [ ]:
def extract_key_frames(pose: Pose, output_dir: str, num_frames: int = 10):
    """
    استخراج إطارات رئيسية متباعدة بالتساوي
    """
    from pathlib import Path
    Path(output_dir).mkdir(exist_ok=True)
    
    total_frames = pose.body.data.shape[0]
    step = total_frames // num_frames
    
    pose_copy = pose.copy()
    pose_copy.focus()
    visualizer = PoseVisualizer(pose_copy)
    
    all_frames = list(visualizer.draw())
    
    for i in range(num_frames):
        frame_idx = i * step
        if frame_idx < len(all_frames):
            output_path = f"{output_dir}/frame_{frame_idx:04d}.png"
            visualizer.save_frame(output_path, all_frames[frame_idx])
    
    print(f"✅ تم استخراج {num_frames} إطار في: {output_dir}")

# extract_key_frames(pose, "key_frames", num_frames=10)

## 7️⃣ العرض السريع (FastAndUglyPoseVisualizer)

In [ ]:
def create_fast_visualization(pose: Pose, output_path: str):
    """
    عرض سريع بجودة أقل (grayscale)
    أسرع بكثير للملفات الكبيرة
    """
    pose_copy = pose.copy()
    pose_copy.focus()
    
    # استخدام الـ visualizer السريع
    visualizer = FastAndUglyPoseVisualizer(pose_copy)
    
    # خلفية سوداء، رسم أبيض
    frames = visualizer.draw(
        background_color=0,    # أسود
        foreground_color=255   # أبيض
    )
    
    visualizer.save_video(output_path, frames)
    
    print(f"✅ تم حفظ الفيديو السريع: {output_path}")

# create_fast_visualization(pose, "fast_output.mp4")

## 8️⃣ تخصيص سمك الخطوط والنقاط

In [ ]:
def create_custom_thickness_video(pose: Pose, output_path: str, thickness: int = 3):
    """
    إنشاء فيديو بسمك خطوط مخصص
    
    Parameters:
    -----------
    thickness : int
        سمك الخطوط والنقاط (افتراضي: يُحسب تلقائياً)
    """
    pose_copy = pose.copy()
    pose_copy.focus()
    
    # تحديد السمك
    visualizer = PoseVisualizer(pose_copy, thickness=thickness)
    
    frames = visualizer.draw()
    visualizer.save_video(output_path, frames)
    
    print(f"✅ تم حفظ الفيديو بسمك {thickness}: {output_path}")

# create_custom_thickness_video(pose, "thick_lines.mp4", thickness=5)
# create_custom_thickness_video(pose, "thin_lines.mp4", thickness=1)

## 9️⃣ عرض مكونات محددة فقط

In [ ]:
def visualize_specific_components(pose: Pose, output_path: str, 
                                  components: list):
    """
    عرض مكونات محددة فقط (مثلاً اليدين فقط)
    
    Parameters:
    -----------
    components : list
        قائمة أسماء المكونات مثل:
        ['LEFT_HAND_LANDMARKS', 'RIGHT_HAND_LANDMARKS']
    """
    # استخراج المكونات المطلوبة فقط
    filtered_pose = pose.get_components(components)
    filtered_pose.focus()
    
    visualizer = PoseVisualizer(filtered_pose)
    frames = visualizer.draw()
    visualizer.save_video(output_path, frames)
    
    print(f"✅ تم حفظ الفيديو مع المكونات: {components}")

# مثال: اليدين فقط
# visualize_specific_components(pose, "hands_only.mp4", 
#                               ['LEFT_HAND_LANDMARKS', 'RIGHT_HAND_LANDMARKS'])

# مثال: الجسم فقط
# visualize_specific_components(pose, "body_only.mp4", 
#                               ['POSE_LANDMARKS'])

## 🔟 عرض مقارنة بين poses

In [ ]:
import cv2

def create_side_by_side_video(pose1: Pose, pose2: Pose, output_path: str,
                              labels: tuple = ('Original', 'Processed')):
    """
    إنشاء فيديو مقارنة جنباً إلى جنب
    """
    pose1_copy = pose1.copy()
    pose2_copy = pose2.copy()
    
    pose1_copy.focus()
    pose2_copy.focus()
    
    vis1 = PoseVisualizer(pose1_copy)
    vis2 = PoseVisualizer(pose2_copy)
    
    frames1 = list(vis1.draw())
    frames2 = list(vis2.draw())
    
    # التأكد من تساوي عدد الإطارات
    min_frames = min(len(frames1), len(frames2))
    
    combined_frames = []
    for i in range(min_frames):
        # دمج الإطارات جنباً إلى جنب
        combined = np.hstack([frames1[i], frames2[i]])
        
        # إضافة labels
        h, w = combined.shape[:2]
        cv2.putText(combined, labels[0], (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        cv2.putText(combined, labels[1], (w//2 + 10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
        
        combined_frames.append(combined)
    
    vis1.save_video(output_path, iter(combined_frames))
    print(f"✅ تم حفظ فيديو المقارنة: {output_path}")

# create_side_by_side_video(pose_original, pose_normalized, "comparison.mp4")

## 1️⃣1️⃣ عرض Pose في Jupyter Notebook

In [ ]:
from IPython.display import Image, display, HTML
import base64

def display_pose_in_notebook(pose: Pose, max_frames: int = 50):
    """
    عرض الـ pose مباشرة في Jupyter Notebook كـ GIF
    """
    pose_copy = pose.copy()
    pose_copy.focus()
    
    visualizer = PoseVisualizer(pose_copy)
    frames = list(visualizer.draw(max_frames=max_frames))
    
    # إنشاء GIF في الذاكرة
    gif_bytes = visualizer.save_gif(None, frames)
    
    # تحويل إلى base64 وعرض
    b64 = base64.b64encode(gif_bytes).decode()
    display(HTML(f'<img src="data:image/gif;base64,{b64}">'))

# display_pose_in_notebook(pose)

In [ ]:
def display_frame_in_notebook(pose: Pose, frame_index: int = 0):
    """
    عرض إطار واحد في Notebook
    """
    import io
    from PIL import Image as PILImage
    
    pose_copy = pose.copy()
    pose_copy.focus()
    
    visualizer = PoseVisualizer(pose_copy)
    frames = list(visualizer.draw(max_frames=frame_index + 1))
    
    if frame_index < len(frames):
        # تحويل من BGR إلى RGB
        frame_rgb = cv2.cvtColor(frames[frame_index], cv2.COLOR_BGR2RGB)
        img = PILImage.fromarray(frame_rgb)
        display(img)
    else:
        print(f"❌ الإطار {frame_index} غير موجود")

# display_frame_in_notebook(pose, frame_index=0)

## 1️⃣2️⃣ دالة شاملة للعرض

In [ ]:
class PoseRenderer:
    """
    كلاس شامل لعرض وتصوير Poses
    """
    
    def __init__(self, pose: Pose, normalize: bool = True):
        self.pose = pose.copy()
        if normalize:
            self.pose.focus()
        self.visualizer = PoseVisualizer(self.pose)
    
    def to_video(self, output_path: str, **kwargs):
        """حفظ كفيديو"""
        frames = self.visualizer.draw(**kwargs)
        self.visualizer.save_video(output_path, frames)
        return output_path
    
    def to_gif(self, output_path: str, max_frames: int = 100, **kwargs):
        """حفظ كـ GIF"""
        frames = list(self.visualizer.draw(max_frames=max_frames, **kwargs))
        self.visualizer.save_gif(output_path, frames)
        return output_path
    
    def to_frames(self, output_dir: str, num_frames: int = 10):
        """استخراج إطارات"""
        from pathlib import Path
        Path(output_dir).mkdir(exist_ok=True)
        
        all_frames = list(self.visualizer.draw())
        step = max(1, len(all_frames) // num_frames)
        
        paths = []
        for i in range(0, len(all_frames), step):
            path = f"{output_dir}/frame_{i:04d}.png"
            self.visualizer.save_frame(path, all_frames[i])
            paths.append(path)
        
        return paths
    
    def overlay_on_video(self, video_path: str, output_path: str, blur: bool = False):
        """رسم على فيديو"""
        frames = self.visualizer.draw_on_video(video_path, blur=blur)
        self.visualizer.save_video(output_path, frames)
        return output_path
    
    def get_gif_bytes(self, max_frames: int = 50) -> bytes:
        """GIF في الذاكرة"""
        frames = list(self.visualizer.draw(max_frames=max_frames))
        return self.visualizer.save_gif(None, frames)

# استخدام:
# renderer = PoseRenderer(pose)
# renderer.to_video("output.mp4")
# renderer.to_gif("output.gif")
# renderer.to_frames("frames_dir")

## 📚 ملخص

| الدالة | الاستخدام |
|--------|----------|
| `visualizer.save_video()` | حفظ كفيديو MP4 |
| `visualizer.save_gif()` | حفظ كـ GIF متحرك |
| `visualizer.save_png()` | حفظ كـ PNG (يدعم الشفافية) |
| `visualizer.save_frame()` | حفظ إطار واحد |
| `visualizer.draw()` | رسم على خلفية ملونة |
| `visualizer.draw_on_video()` | رسم على فيديو |

### CLI
```bash
visualize_pose -i input.pose -o output.mp4 --normalize
```

### الخطوة التالية
انتقل للـ Notebook التالي لتعلم **الميزات المتقدمة** مثل التطبيع والتضخيم!